In [1]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AutoModel, AutoTokenizer
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

## Data Processing

In [4]:
for i in range(10):
    df_csv = pd.read_csv('D:/reviewsData/filtered{}.csv'.format(i)).drop(['Unnamed: 0'], axis=1)
    if i > 0:
        df_csv.index = df_csv.index + len(df)
        df = pd.concat([df,df_csv])
    else:
        df = df_csv

In [5]:
df_recommended_false = df[df['recommended'] == False]['review']
recommended_false = df_recommended_false[df_recommended_false.str.len() > 10]
df_recommended_true = df[df['recommended'] == True]['review']
recommended_true = df_recommended_true[df_recommended_true.str.len() > 10]

In [6]:
df_false = pd.DataFrame(recommended_false, columns=['review'])
df_true = pd.DataFrame(recommended_true, columns=['review'])
df_false['label'] = [0]*len(recommended_false)
df_true['label'] = [1]*len(recommended_true)

In [7]:
df_train = pd.concat([df_true.loc[df_true.index[:len(df_false)],:],df_false],axis=0,ignore_index=True)
train_text, test_text, trainY, testY = train_test_split(df_train['review'].values, df_train['label'].values, test_size=0.2, shuffle = True)

## Model preparation

In [8]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
trainX = tokenizer(train_text[0:50000].tolist(), truncation=True, padding=True, max_length=512)
testX = tokenizer(test_text[0:10000].tolist(), truncation=True, padding=True, max_length=512)

In [10]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

train_dataset = ReviewDataset(trainX, trainY[:50000])
test_dataset = ReviewDataset(testX, testY[:10000])

In [11]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [12]:
training_args = TrainingArguments(
    output_dir='D:/reviewsData/result',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=500,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [13]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [14]:
trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 37500


Step,Training Loss,Validation Loss,Accuracy
500,0.532600,1.341116,0.737500
1000,0.538700,0.448723,0.874500
1500,0.413800,0.428441,0.896600
2000,0.426000,0.430204,0.905600
2500,0.407200,0.405596,0.897900
3000,0.399500,0.273035,0.906800
3500,0.418600,0.335318,0.916400
4000,0.393500,0.486009,0.889300
4500,0.388200,0.374801,0.877100
5000,0.375900,0.386854,0.903100


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-500
Configuration saved in D:/reviewsData/result\checkpoint-500\config.json
Model weights saved in D:/reviewsData/result\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-1000
Configuration saved in D:/reviewsData/result\checkpoint-1000\config.json
Model weights saved in D:/reviewsData/result\checkpoint-1000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-1500
Configuration saved in D:/reviewsData/result\checkpoint-1500\config.json
Model weights saved in D:/reviewsData/result\checkpoint-1500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-2000
Con

Configuration saved in D:/reviewsData/result\checkpoint-14500\config.json
Model weights saved in D:/reviewsData/result\checkpoint-14500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-15000
Configuration saved in D:/reviewsData/result\checkpoint-15000\config.json
Model weights saved in D:/reviewsData/result\checkpoint-15000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-15500
Configuration saved in D:/reviewsData/result\checkpoint-15500\config.json
Model weights saved in D:/reviewsData/result\checkpoint-15500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 4
Saving model checkpoint to D:/reviewsData/result\checkpoint-16000
Configuration saved in D:/reviewsData/result\checkpoint-16000\config.json
Model weights saved in D:/reviewsData/result\checkpoi

RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:274] . unexpected pos 44224 vs 44142

In [ ]:
trainer.evaluate()

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "10000-reviews-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [ ]:
# Example #1
text = """
The first thing is first. 
If you purchase a Macbook, you should not encounter performance issues that will prevent you from learning to code efficiently.
However, in the off chance that you have to deal with a slow computer, you will need to make some adjustments. 
Having too many background apps running in the background is one of the most common causes. 
The same can be said about a lack of drive storage. 
For that, it helps if you uninstall xcode and other unnecessary applications, as well as temporary system junk like caches and old backups.
"""
print(get_prediction(text))

In [16]:
import numpy as np
import torch
token_frequency = {'a':11, 'b':1}
token_feature = {'a':torch.tensor([1,2,3]), 'b':torch.tensor([3,2,1])}
token_feature_list = token_feature.values()
token_feature_list = np.array([item.cpu().detach().numpy() for item in token_feature_list])
token_feature_list[token_frequency > 10]

TypeError: '>' not supported between instances of 'dict' and 'int'